In [1]:
import pandas as pd

In [3]:
cb = pd.read_excel('Data/231013_Customer_Base.xlsx')
ti = pd.read_excel('Data/231013_Transactions_Input.xlsx')

In [4]:
transactions_all = pd.merge(cb, ti, on='customer_id')

In [5]:
transactions_all.columns = map(str.lower, transactions_all.columns)
transactions_all['date'] = pd.to_datetime(transactions_all['date'])

In [6]:
frauds = pd.read_excel('data/231013_Fraud_Cases.xlsx')
frauds.columns = map(str.lower, frauds.columns)

In [12]:
transactions = transactions_all.loc[(transactions_all['in_or_out'] == 'paid_out') & (transactions_all['category'] != 'Cash Withdrawal')]

In [13]:
transactions['bank_from'].unique()

array(['bank_E', 'bank_D', 'bank_C', 'bank_A', 'bank_B'], dtype=object)

In [14]:
transactions

,customer_id,home_bank,customer_prob,transaction_id,description,amount,category,date,month,type,in_or_out,bank_to,bank_from,transac_prob,description_prob,priority
0,10298,bank_E,0.49,2,Online Retailer - Electronics Purchase,199.99,Online Shopping,2023-10-01,October,spending,paid_out,bank_A,bank_E,0.78,0.25,3
1,10298,bank_E,0.49,3,Utilities Payment - Internet Service,60.00,Utilities,2023-10-01,October,spending,paid_out,bank_C,bank_E,0.30,0.22,1
3,10298,bank_E,0.49,33,Rent Payment,1200.00,Housing,2023-10-01,October,spending,paid_out,bank_E,bank_E,0.65,0.62,3
4,10298,bank_E,0.49,2869,Phone Bill Payment - Wireless Provider,70.00,Utilities,2023-10-02,October,spending,paid_out,bank_A,bank_E,0.48,0.34,1
6,10298,bank_E,0.49,2871,Gas Station - Fuel Purchase,40.75,Transportation,2023-10-02,October,spending,paid_out,bank_E,bank_E,0.12,0.28,1


In [53]:
days = list(transactions['date'].unique())
transactions_d1 = transactions[transactions['date'] == days[1]]

In [57]:
transaction_ids = list(transactions_d1.index)
len(transaction_ids)

1323

## The Model

In [70]:
# List containing banks from A to E
banks = transactions['bank_from'].unique()

# Dictionary denoting the internal investigator team size for each bank
team_size = {
    'bank_A' : 8,
    'bank_B' : 12,
    'bank_C' : 10,
    'bank_D' : 10,
    'bank_E' : 10,
}

# Dictionary denoting the fixed cost for hiring an external investigator for each priority level
hire_cost = {
    1 : 40,
    2 : 60,
    3 : 100,
    4 : 150
}

investigation_count = {
    1 : 0.25,
    2 : 0.5,
    3 : 1,
    4 : 2,
}

In [76]:
def solve_for_day(transactions, w1=0.33, w2=0.33, w3=0.33):    

    # Decision Variables
    xi = LpVariable.dicts("x", transactions.index, 0, 1, LpBinary)
    yi = LpVariable.dicts("y", transactions.index, 0, 1, LpBinary)
    
    # Initialize the problem
    prob = LpProblem("Minimize_Cost", LpMinimize)
    
    # Objective Function
    def calculate_objective(row):
        Pr_i = w1 * row['customer_prob'] + w2 * row['transac_prob'] + w3 * row['description_prob']
        F_i = hire_cost[row['priority']]
        k_i = investigation_count[row['priority']]
        return (1 - xi[row.name]) * row['amount'] * k_i * Pr_i + yi[row.name] * F_i - xi[row.name] * row['amount'] * k_i * Pr_i
    
    Z = lpSum(transactions.apply(calculate_objective, axis=1))
    
    prob += Z
    
    # Constraints
    
    # Capacity Constraint for each bank
    for bank in banks:
        T_j = transactions[transactions['bank_from'] == bank].index
        prob += lpSum([xi[i] for i in T_j]) <= team_size[bank]
    
    # Investigator Constraint for each transaction
    for i in transactions.index:
        prob += xi[i] + yi[i] <= 1
    
    # Solve the problem
    prob.solve()
    
    # Find the transaction id's for the investigated transactions
    internally_investigated, externally_investigated = [], []
    for i in transactions.index:
        if xi[i].varValue == 1:
            internally_investigated.append(i)
        elif yi[i].varValue == 1:
            externally_investigated.append(i)
            
    return internally_investigated, externally_investigated

In [82]:
ii, ei = solve_for_day(transactions_d1)
transactions_d1['is_fraud'] = transactions_d1['transaction_id'].isin(frauds['transaction_id']).astype(int)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/_g/3jdpr0d14q1g62hkp6vf5mlh0000gn/T/79259bfee1c64b52a554ab9ebcc9e512-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/_g/3jdpr0d14q1g62hkp6vf5mlh0000gn/T/79259bfee1c64b52a554ab9ebcc9e512-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 1333 COLUMNS
At line 13241 RHS
At line 14570 BOUNDS
At line 17217 ENDATA
Problem MODEL has 1328 rows, 2646 columns and 3969 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is -6020.97 - 0.00 seconds
Cgl0004I processed model has 5 rows, 1323 columns (1323 integer (1323 of which binary)) and 1323 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -6020.97
Cbc0038I Before mini br

/var/folders/_g/3jdpr0d14q1g62hkp6vf5mlh0000gn/T/ipykernel_86387/395774376.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transactions_d1['is_fraud'] = transactions_d1['transaction_id'].isin(frauds['transaction_id']).astype(int)


In [83]:
transactions_d1[transactions_d1['is_fraud'] == 1]

,customer_id,home_bank,customer_prob,transaction_id,description,amount,category,date,month,type,in_or_out,bank_to,bank_from,transac_prob,description_prob,priority,is_fraud
10691,15838,bank_E,0.50,3692,Traffic Violation Fine,124.15,Utilities,2023-10-02,October,spending,paid_out,bank_D,bank_E,0.68,0.68,2,1
189803,18942,bank_B,0.83,2544,Online Gift Purchase for My New Love,69.80,Personal care,2023-10-02,October,spending,paid_out,bank_D,bank_B,0.42,0.63,1,1
189804,18942,bank_B,0.83,2656,Neiman Marcus - Designer Shopping,43.40,Shopping,2023-10-02,October,spending,paid_out,bank_D,bank_B,0.29,0.63,1,1
200300,10796,bank_E,0.65,2440,ClothesOnline additional posting payment,28.75,Shopping,2023-10-02,October,spending,paid_out,bank_C,bank_E,0.37,0.33,1,1
221448,10672,bank_B,0.71,2655,eBay securitng item payment,12.60,Shopping,2023-10-02,October,spending,paid_out,bank_E,bank_B,0.18,0.47,1,1
253574,12325,bank_B,0.73,2883,IRS Tax Payment - Urgent Action Required,82.25,Transfers,2023-10-02,October,spending,paid_out,bank_B,bank_B,0.44,0.40,2,1


## Constraint for Bank Capacity
$\sum_{i \isin \text{Transactions from Bank A} \text{time it takes for transaction i] * x_i  < 12$